statsmodelsの相互相関の検証

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
df_power = pd.read_csv('http://www.tepco.co.jp/forecast/html/images/juyo-2013.csv', skiprows=3, names=['date', 'time', 'actual'], encoding='Shift_JIS')

In [3]:
df_power.head()

,date,time,actual
0,2013/1/1,0:00,2873
1,2013/1/1,1:00,2716
2,2013/1/1,2:00,2592
3,2013/1/1,3:00,2482
4,2013/1/1,4:00,2412


In [4]:
df_power.index = pd.to_datetime(df_power['date'] + ' ' + df_power['time'])

In [5]:
df_power = df_power.drop(['date', 'time'], axis=1)

In [6]:
df_power_daily = df_power.resample('D').max()

In [7]:
df_power_daily.head()

,actual
2013-01-01,3132
2013-01-02,3106
2013-01-03,3558
2013-01-04,4016
2013-01-05,4089


In [8]:
df_temp = pd.read_csv('../data/tokyo_temp.csv', skiprows=5, encoding='Shift_JIS')

In [9]:
df_temp.index = idx_temp = pd.to_datetime(df_temp['Unnamed: 0'])
df_temp = df_temp.drop(['Unnamed: 0', '品質情報', '均質番号', '品質情報.1', '均質番号.1'], axis=1)
df_temp.columns = ['max', 'min']
df_temp.index.name = 'date'

In [10]:
df_temp.head()

,max,min
date,,
2013-01-01,9.6,3.0
2013-01-02,14.4,4.2
2013-01-03,9.9,1.8
2013-01-04,6.9,1.0
2013-01-05,4.8,0.2


In [11]:
len(df_temp), len(df_power_daily)

(206, 365)

In [14]:
df_new = pd.concat([df_temp, df_power_daily.iloc[:len(df_temp)]], axis=1)

In [15]:
df_new.head()

,max,min,actual
2013-01-01,9.6,3.0,3132
2013-01-02,14.4,4.2,3106
2013-01-03,9.9,1.8,3558
2013-01-04,6.9,1.0,4016
2013-01-05,4.8,0.2,4089


In [16]:
df_new.tail()

,max,min,actual
2013-07-21,29.0,21.0,3483
2013-07-22,30.4,23.5,4187
2013-07-23,35.2,25.2,4603
2013-07-24,27.1,24.3,3981
2013-07-25,29.3,24.1,4033


相互相関を計算する

In [77]:
nlags = 30
x_mean = x.mean()
y_mean = y.mean()

ccf_np = [np.mean((x - x_mean) * (y - y_mean)) / (np.std(x)*np.std(y))]
for i in range(1, nlags+1):
    # xと　i時点前のyの相互相関を計算
    _ccf = np.mean((x[i:] - x_mean) * (y[:-i] - y_mean)) / (np.std(x)*np.std(y))
    ccf_np.append(_ccf)

ccf_xy =  sm.tsa.ccf(x, y)[:31]
ccf_np = np.array(ccf_np)
print(ccf_xy)
print(ccf_np)

[-0.32979868 -0.31116132 -0.29658975 -0.2994282  -0.30128682 -0.31436546
 -0.34079486 -0.35146011 -0.37128077 -0.38510173 -0.38960407 -0.39223166
 -0.40080683 -0.38947876 -0.38534167 -0.40924213 -0.44582121 -0.46185581
 -0.48502677 -0.49586986 -0.50203242 -0.49869879 -0.48857328 -0.49732349
 -0.50901217 -0.52628364 -0.54439936 -0.56237884 -0.56299461 -0.55544652
 -0.56674342]
[-0.32979868 -0.31116132 -0.29658975 -0.2994282  -0.30128682 -0.31436546
 -0.34079486 -0.35146011 -0.37128077 -0.38510173 -0.38960407 -0.39223166
 -0.40080683 -0.38947876 -0.38534167 -0.40924213 -0.44582121 -0.46185581
 -0.48502677 -0.49586986 -0.50203242 -0.49869879 -0.48857328 -0.49732349
 -0.50901217 -0.52628364 -0.54439936 -0.56237884 -0.56299461 -0.55544652
 -0.56674342]


In [78]:
ccf_np2 = [np.mean((x - x_mean) * (y - y_mean)) / (np.std(x)*np.std(y))]
for i in range(1, nlags+1):
    # xと　i時点後のyの相互相関を計算
    _ccf = np.mean((x[:-i] - x_mean) * (y[i:] - y_mean)) / (np.std(x)*np.std(y))
    ccf_np2.append(_ccf)

ccf_xy =  sm.tsa.ccf(y, x)[:31]    
ccf_np2 = np.array(ccf_np2)
print(ccf_xy)
print(ccf_np2)

[-0.32979868 -0.31064893 -0.30302941 -0.3031257  -0.28233516 -0.28174299
 -0.29470576 -0.29782515 -0.28743341 -0.28487568 -0.27647852 -0.26121676
 -0.23530887 -0.22073873 -0.19105179 -0.18622102 -0.18602166 -0.15947057
 -0.16002456 -0.15965119 -0.14338357 -0.12573765 -0.10828607 -0.08572854
 -0.06858216 -0.06181091 -0.06334083 -0.07322346 -0.0578149  -0.03507868
 -0.02729947]
[-0.32979868 -0.31064893 -0.30302941 -0.3031257  -0.28233516 -0.28174299
 -0.29470576 -0.29782515 -0.28743341 -0.28487568 -0.27647852 -0.26121676
 -0.23530887 -0.22073873 -0.19105179 -0.18622102 -0.18602166 -0.15947057
 -0.16002456 -0.15965119 -0.14338357 -0.12573765 -0.10828607 -0.08572854
 -0.06858216 -0.06181091 -0.06334083 -0.07322346 -0.0578149  -0.03507868
 -0.02729947]


sm.tsa.ccf(x, y) -> xと過去のyとの相互相関を求めている  
sm.tsa.ccf(y, x) -> xと未来のyとの相互相関を求めている